In [104]:
#  dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [105]:
# reading in beer ratings csv

df=pd.read_csv(r'data\beer_rating.csv')
df.head()


,beer name,id,beer type,Name,brewery,Description,ABV,Ave Rating,Min IBU,Max IBU,...,Body,Alcohol,Bitter,Sweet,Sour,Salty,Fruits,Hoppy,Spices,Malty
0,Amber,251,Altbier,8,Alaskan Brewing Co.,"Notes:Richly malty and long on the palate, wit...",5.3,3.65,25,50,...,32,9,47,74,33,0,33,57,8,111
1,Double Bag,252,Altbier,8,Long Trail Brewing Co.,"Notes:This malty, full-bodied double alt is al...",7.2,3.90,25,50,...,57,18,33,55,16,0,24,35,12,84
2,Long Trail Ale,253,Altbier,8,Long Trail Brewing Co.,Notes:Long Trail Ale is a full-bodied amber al...,5.0,3.58,25,50,...,37,6,42,43,11,0,10,54,4,62
3,Doppelsticke,254,Altbier,8,Uerige Obergärige Hausbrauerei,Notes:,8.5,4.15,25,50,...,55,31,47,101,18,1,49,40,16,119
4,Scurry,255,Altbier,8,Off Color Brewing,Notes:Just cause it's dark and German doesn't ...,5.3,3.67,25,50,...,69,10,63,120,14,0,19,36,15,218


In [106]:
#  read in usbreweries csv
brdf = pd.read_csv(r'data\usbreweries.csv')
brdf.head()

,Unnamed: 0,obdb_id,name,brewery_type,street,city,state,postal_code,website_url,phone,country,longitude,latitude
0,0,siluria-brewing-company-alabaster,Siluria Brewing Company,planning,NaN,Alabaster,Alabama,35007-8501,http://www.siluriabrewing.com,2054827661,United States,-86.816377,33.244281
1,1,cheaha-brewing-co-anniston,Cheaha Brewing Co,brewpub,1208 Walnut Ave,Anniston,Alabama,36201-4526,http://www.cheahabrewingcompany.com,2567707300,United States,-85.833774,33.660167
2,2,avondale-brewing-co-birmingham,Avondale Brewing Co,micro,201 41st St S,Birmingham,Alabama,35222-1932,http://www.avondalebrewing.com,2057775456,United States,-86.774322,33.524521
3,3,birmingham-district-brewing-birmingham,Birmingham District Brewing,planning,NaN,Birmingham,Alabama,35242-5651,NaN,6019380486,United States,-86.802433,33.520682
4,4,brewpub-in-planning-birmingham-birmingham,Brewpub In Planning - Birmingham,planning,NaN,Birmingham,Alabama,35222,NaN,2565854531,United States,-86.802433,33.520682


In [112]:
# creating brewery df based on the beer ratings csv to create a universaL index for all breweries
df = df[['brewery']].drop_duplicates()
ratings_brew_df = pd.DataFrame(df.groupby('brewery'))
ratings_brewery_df = pd.DataFrame()
ratings_brewery_df['brewery_name'] = ratings_brew_df[0]
ratings_brewery_df.head()


,brewery_name
0,(512) Brewing Company
1,10 Barrel Brewing Co.
2,14th Star Brewing Co.
3,18th Street Brewery - Gary Taproom
4,1940's Brewing Company


In [115]:
# df based on open brewery db, to create a universal index for all breweries
bdf = pd.read_csv(r'data/usbreweries.csv')
bdf = bdf[['name']].drop_duplicates()
bdf = pd.DataFrame(bdf.groupby('name'))

grouped_bdf = pd.DataFrame()

grouped_bdf['brewery_name'] = bdf[0]

# grouped_bdf['brewery_name'] = grouped_bdf['brewery_name'].str.replace('(', '')
# grouped_bdf['brewery_name'] = grouped_bdf['brewery_name'].str.replace(')', '')

grouped_bdf.reset_index()

,index,brewery_name
0,0,(405) Brewing Co
1,1,10 Barrel Brewing Co
2,2,10 Barrel Brewing Co - Bend Pub
3,3,10 Barrel Brewing Co - Boise
4,4,10 Barrel Brewing Co - Denver
...,...,...
5014,5014,ZwanzigZ Brewing
5015,5015,Zydeco Brew Werks
5016,5016,Zymurcracy Beer Company
5017,5017,Zymurgy Brew Works


In [122]:
# merge based on brewery name

merged_df = pd.merge(ratings_brewery_df, grouped_bdf, on=['brewery_name'],how='outer')
merged_df = merged_df.drop_duplicates('brewery_name').reset_index().rename(columns={'index':'brewery_id'})
merged_df.head()


,brewery_id,brewery_name
0,0,(512) Brewing Company
1,1,10 Barrel Brewing Co.
2,2,14th Star Brewing Co.
3,3,18th Street Brewery - Gary Taproom
4,4,1940's Brewing Company


In [120]:
# create beer : brewery df
rdf = pd.read_csv(r'data/beer_rating.csv')
rdf = rdf[['id','beer name','brewery']]
rdf[['beer_id','beer_name','brewery_name']] = rdf[['id','beer name','brewery']]

beer_to_brewery_df = pd.merge(rdf,merged_df,on=['brewery_name'])
beer_to_brewery_df['brewery'] = merged_df['brewery_id']
beer_to_brewery_df = beer_to_brewery_df.drop(columns=['brewery'])
beer_to_brewery_df = beer_to_brewery_df[['brewery_name','brewery_id','beer_name','beer_id']]
beer_to_brewery_df.head()

id_df = beer_to_brewery_df[['brewery_id','beer_id']]
id_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5558 entries, 0 to 5557
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   brewery_id  5558 non-null   int64
 1   beer_id     5558 non-null   int64
dtypes: int64(2)
memory usage: 130.3 KB


In [123]:
# create brewery info df
bdf = pd.read_csv(r'data/usbreweries.csv')
bdf = bdf.drop(columns=['Unnamed: 0','phone','country'])

brewery_info_df = bdf.rename(columns={'name':'brewery_name'})
brewery_info_df.head()

brewery_info_df = pd.merge(brewery_info_df, merged_df, on=['brewery_name'],how='outer')

brewery_info_df.head()

,obdb_id,brewery_name,brewery_type,street,city,state,postal_code,website_url,longitude,latitude,brewery_id
0,siluria-brewing-company-alabaster,Siluria Brewing Company,planning,NaN,Alabaster,Alabama,35007-8501,http://www.siluriabrewing.com,-86.816377,33.244281,5410
1,cheaha-brewing-co-anniston,Cheaha Brewing Co,brewpub,1208 Walnut Ave,Anniston,Alabama,36201-4526,http://www.cheahabrewingcompany.com,-85.833774,33.660167,2533
2,avondale-brewing-co-birmingham,Avondale Brewing Co,micro,201 41st St S,Birmingham,Alabama,35222-1932,http://www.avondalebrewing.com,-86.774322,33.524521,1793
3,birmingham-district-brewing-birmingham,Birmingham District Brewing,planning,NaN,Birmingham,Alabama,35242-5651,NaN,-86.802433,33.520682,2063
4,brewpub-in-planning-birmingham-birmingham,Brewpub In Planning - Birmingham,planning,NaN,Birmingham,Alabama,35222,NaN,-86.802433,33.520682,2304


In [128]:
brewery_info_df.reindex(columns=['brewery_id','brewery_name','brewery_type','city','state','country','latitude','longitude'])
brewery_info_df.sort_values(by=['brewery_id'])

,obdb_id,brewery_name,brewery_type,street,city,state,postal_code,website_url,longitude,latitude,brewery_id
5084,NaN,(512) Brewing Company,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
5085,NaN,10 Barrel Brewing Co.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
5086,NaN,14th Star Brewing Co.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
5087,NaN,18th Street Brewery - Gary Taproom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
5088,NaN,1940's Brewing Company,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...
1625,zwanzigz-brewing-columbus,ZwanzigZ Brewing,brewpub,1038 Lafayette Ave,Columbus,Indiana,47201-5756,http://www.zwanzigz.com,-85.918836,39.208657,6329
1306,zydeco-brew-werks-tampa,Zydeco Brew Werks,brewpub,1902 E 7th Ave,Tampa,Florida,33605-3810,NaN,-82.437832,27.960344,6330
4057,zymurcracy-beer-company-rapid-city,Zymurcracy Beer Company,micro,4624 Creek Dr Ste 6,Rapid City,South Dakota,57701-2165,http://www.zymurcracybeer.com,-103.179520,44.038115,6331
774,zymurgy-brew-works-torrance,Zymurgy Brew Works,micro,22755 Hawthorne Blvd,Torrance,California,90505-3613,http://www.zymurgybrewworks.com,-118.352467,33.863537,6332


In [53]:
# creating id numbers for breweries without id numbers
new_ids = np.arange(6334,6372)
# new_id_df = pd.DataFrame(new_ids)

# brewery_info_df['brewery_id'] = brewery_info_df['brewery_id'].fillna(value = new_ids)

fix_null_ids = brewery_info_df.loc[brewery_info_df['brewery_id'].isnull()]
fix_null_ids = fix_null_ids[['brewery_name','brewery_id']]
fix_null_ids['brewery_id'] = new_ids.tolist()
fix_null_ids.head(40)

brewery_info_df = pd.merge(brewery_info_df, fix_null_ids, on=['brewery_name'], how='inner')
brewery_info_df.head()
# brewery_info_df = pd.merge(brewery_info_df, fix_null_ids, on=['brewery_name'],how='outer')
# brewery_info_df.tail()
# brewery_info_df.sort_values(by=['brewery_id']).tail()

,brewery_name,brewery_type,street,city,state,website_url,longitude,latitude,brewery_id_x,brewery_id_y
0,Moonraker Brewing Company (Production Facility),micro,2349 Rickenbacker Way,Auburn,California,NaN,-121.081369,38.952499,NaN,6334
1,The Brew On Broadway (The BoB),micro,3445 S Broadway,Englewood,Colorado,http://www.thebrewonbroadway.com,-104.987720,39.654168,NaN,6335
2,Granite City Food & Brewery (#21),brewpub,230 Conference Center Dr,East Peoria,Illinois,http://www.gcfb.com,-89.589836,40.683059,NaN,6336
3,Granite City Food & Brewery (#30),brewpub,1828 Abriter Ct,Naperville,Illinois,http://www.gcfb.net,-88.129447,41.802839,NaN,6337
4,Granite City Food & Brewery (#22),brewpub,14035 S La Grange Rd,Orland Park,Illinois,http://www.gcfb.com,-87.853891,41.637483,NaN,6338


In [ ]:
# create beer profile df

In [ ]:
# create beer flavor profile df